# Offline Engine API

SGLang provides a direct inference engine without the need for an HTTP server, especially for use cases where additional HTTP server adds unnecessary complexity or overhead. Here are two general use cases:

- Offline Batch Inference
- Custom Server on Top of the Engine

This document focuses on the offline batch inference, demonstrating four different inference modes:

- Non-streaming synchronous generation
- Streaming synchronous generation
- Non-streaming asynchronous generation
- Streaming asynchronous generation

Additionally, you can easily build a custom server on top of the SGLang offline engine. A detailed example working in a python script can be found in [custom_server](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/custom_server.py).



## Nest Asyncio
Note that if you want to use **Offline Engine** in ipython or some other nested loop code, you need to add the following code:
```python
import nest_asyncio

nest_asyncio.apply()

```

## Advanced Usage

The engine supports [vlm inference](https://github.com/sgl-project/sglang/blob/main/examples/runtime/engine/offline_batch_inference_vlm.py) as well as [extracting hidden states](https://github.com/sgl-project/sglang/blob/main/examples/runtime/hidden_states). 

Please see [the examples](https://github.com/sgl-project/sglang/tree/main/examples/runtime/engine) for further use cases.

## Offline Batch Inference

SGLang offline engine supports batch inference with efficient scheduling.

In [1]:
# launch the offline engine
import asyncio

import sglang as sgl
import sglang.test.doc_patch
from sglang.utils import async_stream_and_merge, stream_and_merge

llm = sgl.Engine(model_path="qwen/qwen2.5-0.5b-instruct")

[2026-01-23 17:59:02] INFO utils.py:148: Note: detected 112 virtual cores but NumExpr set to maximum of 64, check "NUMEXPR_MAX_THREADS" environment variable.


[2026-01-23 17:59:02] INFO utils.py:151: Note: NumExpr detected 112 cores but "NUMEXPR_MAX_THREADS" not set, so enforcing safe limit of 16.


[2026-01-23 17:59:02] INFO utils.py:164: NumExpr defaulting to 16 threads.


[2026-01-23 17:59:05] INFO server_args.py:1769: Attention backend not specified. Use fa3 backend by default.


[2026-01-23 17:59:05] INFO server_args.py:2658: Set soft_watchdog_timeout since in CI


...

Capturing batches (bs=128 avail_mem=12.71 GB):   0%|          | 0/20 [00:00<?, ?it/s]

Capturing batches (bs=104 avail_mem=11.63 GB):  15%|█▌        | 3/20 [00:00<00:03,  5.46it/s]

Capturing batches (bs=56 avail_mem=10.64 GB):  45%|████▌     | 9/20 [00:00<00:00, 14.84it/s]

Capturing batches (bs=16 avail_mem=10.60 GB):  60%|██████    | 12/20 [00:01<00:00, 17.76it/s]

Capturing batches (bs=1 avail_mem=7.40 GB): 100%|██████████| 20/20 [00:01<00:00, 15.51it/s]


### Non-streaming Synchronous Generation

In [2]:
prompts = [
    "Hello, my name is",
    "The president of the United States is",
    "The capital of France is",
    "The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

outputs = llm.generate(prompts, sampling_params)
for prompt, output in zip(prompts, outputs):
    print("===============================")
    print(f"Prompt: {prompt}\nGenerated text: {output['text']}")

Prompt: Hello, my name is
Generated text:  Alisher and I'm a software developer who likes to learn. I'm fascinated by the way algorithms can transform the world and that makes me very motivated. I have a passion for programming, and I love to work with different programming languages and tools. I'm an avid reader and writer and enjoy creating my own content. What would you like to know about me? As an AI language model, I don't have personal experiences, emotions, or a physical presence, so I don't have a physical address or a personal connection. However, I can provide you with general information about my capabilities based on the information you provide me. Could
Prompt: The president of the United States is
Generated text:  5 feet 3 inches tall. If someone wants to stand in front of the president while they are in a wheelchair, how tall will the person be?

To determine the height of the person who wants to stand in front of the president while they are in a wheelchair, we need to 

### Streaming Synchronous Generation

In [3]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {
    "temperature": 0.2,
    "top_p": 0.9,
}

print("\n=== Testing synchronous streaming generation with overlap removal ===\n")

for prompt in prompts:
    print(f"Prompt: {prompt}")
    merged_output = stream_and_merge(llm, prompt, sampling_params)
    print("Generated text:", merged_output)
    print()


=== Testing synchronous streaming generation with overlap removal ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is


Generated text:  [Name] and I'm a [job title] at [company name]. I'm excited to meet you and learn more about your career. What can you tell me about yourself? I'm a [job title] at [company name], and I have been working in the [industry] for [number of years] years. I have a passion for [job title] and I'm always looking for ways to [job title] my skills and knowledge. What's your favorite hobby or activity? I love [hobby or activity] and I find it really helps me relax and recharge. What's your favorite book or movie?

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is


Generated text:  Paris, also known as the City of Light. It is a historic city with a rich history and a vibrant culture. The city is known for its beautiful architecture, world-renowned museums, and annual festivals. Paris is also a major center for finance, business, and art. It is a popular tourist destination and a major economic and cultural hub in Europe. The city is home to many famous landmarks such as the Eiffel Tower, Notre-Dame Cathedral, and the Louvre Museum. Paris is a city that has a unique blend of old-world charm and modernity, making it a popular destination for both locals and tourists alike.

Prompt: Explain possible future trends in artificial intelligence. The future of AI is


Generated text:  likely to be characterized by rapid advancements in areas such as machine learning, natural language processing, and computer vision. These technologies will continue to improve and become more integrated into our daily lives, from self-driving cars to personalized healthcare. AI will also continue to play a key role in solving complex problems such as climate change and global health crises. However, there are also potential risks and challenges associated with AI, including the potential for job displacement and ethical concerns around AI algorithms. As we continue to explore and develop AI, it is important to carefully consider its potential impact on society and work to address any potential risks and challenges.



### Non-streaming Asynchronous Generation

In [4]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous batch generation ===")


async def main():
    outputs = await llm.async_generate(prompts, sampling_params)

    for prompt, output in zip(prompts, outputs):
        print(f"\nPrompt: {prompt}")
        print(f"Generated text: {output['text']}")


asyncio.run(main())


=== Testing asynchronous batch generation ===



Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text:  [Name], and I'm a/an [Age] year old [Occupation]. I'm confident and outgoing, but I'm also humble and curious. My background in [Education/Training] has taught me valuable life lessons and allowed me to develop my unique perspective. I enjoy sports, music, and travel, and I love learning new things. I'm always looking for new experiences and ways to improve myself, and I'm always willing to challenge myself to grow and evolve. I'm a/an [Occupation] with a/an [Skill/Innovation/Passion]. How can I be helpful for [Job or Purpose]?

Prompt: Provide a concise factual statement about France’s capital city. The capital of France is
Generated text:  Paris, which is known as the City of Light. 

This statement provides a concise factual representation of the capital city of France, indicating that it is a significant cultural and historical center. It mentions that Paris is referred t

### Streaming Asynchronous Generation

In [5]:
prompts = [
    "Write a short, neutral self-introduction for a fictional character. Hello, my name is",
    "Provide a concise factual statement about France’s capital city. The capital of France is",
    "Explain possible future trends in artificial intelligence. The future of AI is",
]

sampling_params = {"temperature": 0.8, "top_p": 0.95}

print("\n=== Testing asynchronous streaming generation (no repeats) ===")


async def main():
    for prompt in prompts:
        print(f"\nPrompt: {prompt}")
        print("Generated text: ", end="", flush=True)

        # Replace direct calls to async_generate with our custom overlap-aware version
        async for cleaned_chunk in async_stream_and_merge(llm, prompt, sampling_params):
            print(cleaned_chunk, end="", flush=True)

        print()  # New line after each prompt


asyncio.run(main())


=== Testing asynchronous streaming generation (no repeats) ===

Prompt: Write a short, neutral self-introduction for a fictional character. Hello, my name is
Generated text: 

 [

insert

 character

's

...

 urban

 environments

.

 AI

 will

In [6]:
llm.shutdown()